In [59]:
import pandas as pd 
import pymongo 
from flask import jsonify
import json
import requests


conn='mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
url = "http://bikeshare.metro.net/stations/json/"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers = headers)

In [34]:
data = (json.loads(response.text))["features"]
data_list = []
for row in data:
    data_list.append({"start_station": row["properties"]["kioskId"],
                      "station_name": row["properties"]["name"],
                      "Latitude" : row["properties"]["latitude"],
                      "Longitude": row["properties"]["longitude"],
                      "KioskPublicStatus" : row["properties"]["kioskPublicStatus"],
                      "KioskStatus" : row["properties"]["kioskStatus"],
                      "BikesAvailable" : row["properties"]["bikesAvailable"],
                      "DocksAvailable" : row["properties"]["docksAvailable"],
                      "TotalDocks" : row["properties"]["totalDocks"],
                      "CloseTime" : row["properties"]["closeTime"],
                      "OpenTime" : row["properties"]["openTime"],
                      "TimeZone" : row["properties"]["timeZone"],
                      "Address" : row["properties"]["addressStreet"],
                      "City" : row["properties"]["addressCity"],
                      "State" : row["properties"]["addressState"],
                      "ZipCode" : row["properties"]["addressZipCode"]
                       })

In [35]:
df = pd.DataFrame(data_list)
df.head()

,Address,BikesAvailable,City,CloseTime,DocksAvailable,KioskPublicStatus,KioskStatus,Latitude,Longitude,OpenTime,State,TimeZone,TotalDocks,ZipCode,start_station,station_name
0,723 Flower Street,25,Los Angeles,23:58:00,2,Active,FullService,34.04850,-118.25854,00:02:00,CA,Pacific Standard Time,27,90017,3005,7th & Flower
1,729 S Olive Street,11,Los Angeles,23:58:00,12,Active,FullService,34.04554,-118.25667,00:02:00,CA,Pacific Standard Time,24,90014,3006,Olive & 8th
2,557 S 5th Street,12,Los Angeles,23:58:00,11,Active,FullService,34.05048,-118.25459,00:02:00,CA,Pacific Standard Time,23,90071,3007,5th & Grand
3,865 S Figueroa Street,1,Los Angeles,23:58:00,14,Active,FullService,34.04661,-118.26273,00:02:00,CA,Pacific Standard Time,15,90017,3008,Figueroa & 9th
4,401 East 11th Street,9,Los Angeles,23:58:00,13,Active,FullService,34.03705,-118.25487,00:02:00,CA,Pacific Standard Time,22,90015,3010,11th & Maple


In [36]:
db = client.bike_data_db
collection = db.bike_trip.find()
trips = []
for trip in collection:
    trips.append(trip)
df_users = pd.DataFrame(trips)

In [37]:
df_users.head()

,_id,bike_id,duration,end_day,end_lat,end_lon,end_station,end_time,passholder_type,plan_duration,start_day,start_lat,start_lon,start_station,start_time,trip_id,trip_route_category,weekday
0,5c02ecce16209a0decfee67e,6279,8,2018-07-01,34.046520,-118.237411,3082,00:12:00,Monthly Pass,30,2018-07-01,34.035801,-118.233170,3058,00:04:00,94851140,One Way,Sunday
1,5c02ecce16209a0decfee67f,6518,8,2018-07-01,34.046520,-118.237411,3082,00:12:00,Flex Pass,365,2018-07-01,34.035801,-118.233170,3058,00:04:00,94851141,One Way,Sunday
2,5c02ecce16209a0decfee680,4823,15,2018-07-01,34.165291,-118.150970,4174,00:24:00,Monthly Pass,365,2018-07-01,34.145248,-118.150070,4147,00:09:00,94851138,One Way,Sunday
3,5c02ecce16209a0decfee681,6115,7,2018-07-01,34.147499,-118.148010,4162,00:29:00,Monthly Pass,30,2018-07-01,34.140999,-118.132088,4157,00:22:00,94851137,One Way,Sunday
4,5c02ecce16209a0decfee682,12055,35,2018-07-01,33.779819,-118.263023,3013,00:58:00,Walk-up,1,2018-07-01,33.779819,-118.263023,3013,00:23:00,94851136,Round Trip,Sunday


In [38]:
df_merged = df_users.merge(df, on="start_station", how="inner")

In [39]:
df_merged.head()

,_id,bike_id,duration,end_day,end_lat,end_lon,end_station,end_time,passholder_type,plan_duration,...,KioskPublicStatus,KioskStatus,Latitude,Longitude,OpenTime,State,TimeZone,TotalDocks,ZipCode,station_name
0,5c02ecce16209a0decfee67e,6279,8,2018-07-01,34.046520,-118.237411,3082,00:12:00,Monthly Pass,30,...,Active,FullService,34.0358,-118.23317,00:02:00,CA,Pacific Standard Time,29,90021,Industrial St & Mateo St
1,5c02ecce16209a0decfee67f,6518,8,2018-07-01,34.046520,-118.237411,3082,00:12:00,Flex Pass,365,...,Active,FullService,34.0358,-118.23317,00:02:00,CA,Pacific Standard Time,29,90021,Industrial St & Mateo St
2,5c02ecce16209a0decfee874,6065,20,2018-07-01,34.045540,-118.256668,3006,18:19:00,One Day Pass,1,...,Active,FullService,34.0358,-118.23317,00:02:00,CA,Pacific Standard Time,29,90021,Industrial St & Mateo St
3,5c02ecce16209a0decfee879,6428,14,2018-07-01,34.045540,-118.256668,3006,18:19:00,One Day Pass,1,...,Active,FullService,34.0358,-118.23317,00:02:00,CA,Pacific Standard Time,29,90021,Industrial St & Mateo St
4,5c02ecce16209a0decfee8e8,12243,7,2018-07-01,34.039219,-118.236488,3054,19:53:00,One Day Pass,1,...,Active,FullService,34.0358,-118.23317,00:02:00,CA,Pacific Standard Time,29,90021,Industrial St & Mateo St


In [1]:
from datetime import datetime

In [29]:
for time_sl in df_users["start_time"]:
    time_slices = datetime.strptime(time_sl, "%H:%M:%S").strftime("%H")
    print(time_slices)

00
00
00
00
00
00
00
00
00
00
00
00
00
00
01
01
01
01
01
01
01
01
01
01
02
02
02
02
02
02
02
02
03
04
05
05
06
06
06
06
06
06
07
07
07
07
07
07
07
07
08
08
08
08
08
08
08
08
08
08
08
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
1

20
20
20
20
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
00
00
00
00
00
00
00
00
00
00
01
01
01
01
01
01
01
01
01
01
01
03
04
04
05
05
05
06
06
06
06
06
06
06
06
06
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
0

08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
1

12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
17
17
1

11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
1

...

09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
1

11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
1

11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
1

13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
1

17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
23
23
23
23
2

In [44]:
df_merged["time_slices"] = [datetime.strptime(time_sl, "%H:%M:%S").strftime("%H") for time_sl in df_merged["start_time"]]


In [47]:
df_merged["time_slices"].min()


'00'

In [51]:
df_grouped = df_merged.groupby("time_slices")["duration"].sum()

In [56]:
df_grouped = df_grouped.reset_index()

In [58]:
df_grouped.sort_values("time_slices")

,time_slices,duration
0,00,45894
1,01,34110
2,02,22395
3,03,8574
4,04,6766
5,05,14288
6,06,25443
7,07,56858
8,08,105205
9,09,151834


In [61]:
list(df_grouped)

['time_slices', 'duration']